1. Установите библиотеку python-telegram-bot с помощью команды pip install python-telegram-bot.
2. Создайте бота в Telegram, следуя инструкциям, которые я описал ранее. Получите токен авторизации бота.
3. Создайте новый файл с расширением .py и импортируйте необходимые библиотеки:

In [1]:
import telegram
import logging
import pandas as pd
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters


ImportError: cannot import name 'Filters' from 'telegram.ext' (/Users/kirillkuznecov/opt/anaconda3/envs/ds/lib/python3.11/site-packages/telegram/ext/__init__.py)

In [ ]:
# Для получения списка каналов пользователя используйте метод `getChatMember` с параметром `@me` для получения списка чатов, в которых вы состоите. 
# Затем вы можете отфильтровать список, чтобы получить только каналы.

# Для спарсивания новостей из каналов используйте метод `getUpdates`, чтобы получить список последних обновлений. 
# Затем вы можете отфильтровать список, чтобы получить только обновления из интересующих вас каналов. Для сохранения новостей в датафрейм вы можете использовать библиотеку `pandas`.

# замените <YOUR_TOKEN> на ваш токен бота
bot = telegram.Bot(token='6235316081:AAEV_lK5gloGpAPiQRpfbS42cgYZfm2xjmo')

# получаем список чатов, в которых состоит пользователь
chats = bot.getChatMember(chat_id='@me')

# отбираем только каналы
channels = [chat for chat in chats if chat.type == 'channel']

# список интересующих каналов
interesting_channels = ['@channel1', '@channel2']

# получаем последние обновления
updates = bot.getUpdates()

# отбираем только обновления из интересующих каналов
interesting_updates = [update for update in updates if update.channel_post and update.channel_post.chat.username in interesting_channels]

# сохраняем новости в датафрейм
news = pd.DataFrame(columns=['date', 'text'])
for update in interesting_updates:
    date = update.channel_post.date
    text = update.channel_post.text
    news = news.append({'date': date, 'text': text}, ignore_index=True)


4. Настройте логирование:

In [ ]:
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)
logger = logging.getLogger(__name__)


5. Создайте функцию для обработки команды / start:

In [ ]:
def start(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, 
                             text="Привет! Я бот для сбора новостей. Отправь мне ссылку на новостной канал в Telegram.")

6. Создайте функцию для обработки сообщений с ссылкой на канал:

In [ ]:
def get_news(update, context):
    channel_link = update.message.text
    channel = context.bot.get_chat(chat_id=channel_link)
    messages = context.bot.get_chat_messages(chat_id=channel_link)
    news_list = []
    for message in messages:
        news_list.append([message.date, message.text])
    df = pd.DataFrame(news_list, columns=["Дата", "Новость"])
    df.to_excel("news.xlsx")
    context.bot.send_message(
        chat_id=update.effective_chat.id, text="Новости сохранены в файл news.xlsx")


7. Создайте функцию main для запуска бота:

In [ ]:
def main():
    updater = Updater(token=TOKEN, use_context=True)
    dp = updater.dispatcher
    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(MessageHandler(Filters.regex(r"https://t.me/.*"), get_news))
    updater.start_polling()
    updater.idle()


8. Запустите бота, вызвав функцию main().

In [ ]:
if __name__ == 'main':
    main()

Этот бот будет реагировать на команду / start и ожидать от пользователя ссылку на новостной канал в Telegram. После получения ссылки бот соберет все новости из канала и сохранит их в файл news.xlsx.